In [1]:
import pandas  as pd
import numpy as np  



In [2]:
#opties
pd.set_option('display.float_format', '{:.0f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

FUNCTIES

In [3]:
#Stukje code om standaar data in te lezen en colomen op te schonen en te laten zien
def eda_basis(naam, df_naam, skip_rows=None):
    df = pd.read_csv(naam, sep=',', skiprows=skip_rows, low_memory=False)
    df.columns = [c.replace(" ", "_").lower() for c in df.columns]

    unieke_waarden = df.nunique()
    aantal_nan = df.isna().sum()
    percent_nan = (aantal_nan / len(df) * 100).round(2)
    kolomtypes = df.dtypes
    
    overzicht = pd.DataFrame({
        'type': kolomtypes,
        'unieke_waarden': unieke_waarden,
        'aantal_NaN': aantal_nan,
        'percentage_NaN': percent_nan.astype(str) + '%'
    })
    
    globals()[df_naam] = df
    return #display(overzicht)



In [4]:
def verwerk_urenbestand(pad):
    #fix
    df = pd.read_csv(pad, sep=',')
    df.columns = [c.replace(" ", "_").lower() for c in df.columns]

    #filt
    df = df.drop(columns=[
        "hoofdrelatie_naam", "relatie_naam", "kantoormedewerker_hoofdrelatie",
        "medewerker", "type_uren", "uren", "tarief", "uren_bedrag",
    ])
    df = df[~df["relatie_nummer"].isin([10, 31113106, 40001010, 878701, 40005371, 40004040, -3])]
    df = df.merge(df_maatschappij_subbranche, left_on='gekoppelde_polis', right_on='polis', how='left')

    #dates
    df['datum'] = pd.to_datetime(df['datum'], format="%d-%m-%Y", errors='coerce')
    df['jaar'] = df['datum'].dt.year
    #df['subbranche'] = df['subbranche'].fillna('(Onbekend)')
    #df['coassurantiemaatschappij_soort'] = df['coassurantiemaatschappij_soort'].fillna('(Onbekend)')
    print(df.columns)
    #basics
    result = df.groupby(['relatie_nummer', 'jaar']).agg(
        totaal_minuten               = ('minuten', 'sum'),
        frequentie_taakregistratie   = ('minuten', 'count'),
        schade                       = ('schadegeval', 'nunique'),
        unieke_werksoort             = ('werksoort', 'nunique'),
        unieke_dienst                = ('dienst', 'nunique'),
        unieke_gekoppeld_product     = ('gekoppelde_product_\t', 'nunique'),
        unieke_soort_dvo             = ('soort_dvo', 'nunique'),
        unieke_afdeling_dvo          = ('afdeling_dvo', 'nunique'),
        unieke_soort_uren            = ('soort_uren', 'nunique')
    ).reset_index()

    #freq and mintus
    def maak_minuten_en_freq(df, col):
        df_cat = df[df[col].notna()]
        minuten = df_cat.pivot_table(
            index=['relatie_nummer', 'jaar'],
            columns=col,
            values='minuten',
            aggfunc='sum',
            fill_value=0
        ).reset_index()
        minuten.columns.name = None
        minuten = minuten.rename(columns=lambda x: f"minuten_{col}_{x}" if x not in ['relatie_nummer', 'jaar'] else x)
        freq = pd.crosstab(
            index=[df_cat['relatie_nummer'], df_cat['jaar']],
            columns=df_cat[col]
        ).reset_index()
        freq.columns.name = None
        freq = freq.rename(columns=lambda x: f"freq_{col}_{x}" if x not in ['relatie_nummer', 'jaar'] else x)
        freq = freq.fillna(0)
        return minuten.merge(freq, on=['relatie_nummer', 'jaar'], how='outer')

    #uniqes
    def maak_unieke_polissen_per_categorie(df, col):
        df_cat = df[df[col].notna()]
        unieke_polissen = df_cat.groupby(['relatie_nummer', 'jaar', col])['gekoppelde_polis'] \
                              .nunique().reset_index()
        unieke_polissen = unieke_polissen.pivot_table(
            index=['relatie_nummer', 'jaar'],
            columns=col,
            values='gekoppelde_polis',
            fill_value=0
        ).reset_index()
        unieke_polissen.columns.name = None
        unieke_polissen = unieke_polissen.rename(
            columns=lambda x: f"unieke_polissen_{col}_{x}" if x not in ['relatie_nummer', 'jaar'] else x
        )
        return unieke_polissen

    categorische_kolommen = ['soort_dvo', 'subbranche', 'coassurantiemaatschappij_soort']
    for kolom in categorische_kolommen:
        features  = maak_minuten_en_freq(df, kolom)
        result    = result.merge(features,  on=['relatie_nummer', 'jaar'], how='left')
        unieke_df = maak_unieke_polissen_per_categorie(df, kolom)
        result    = result.merge(unieke_df, on=['relatie_nummer', 'jaar'], how='left')

    #EWA
    def maak_ewm_minuten_per_polis_subbranche(df, alpha=0.5):
        minuten = df.groupby(['relatie_nummer', 'jaar', 'subbranche'])['minuten'] \
                    .sum().reset_index()
        polissen = df.groupby(['relatie_nummer', 'jaar', 'subbranche'])['gekoppelde_polis'] \
                     .nunique().reset_index(name='aantal_polissen')
        merged = pd.merge(minuten, polissen,
                          on=['relatie_nummer', 'jaar', 'subbranche'],
                          how='left')
        merged['minuten_per_polis'] = merged['minuten'] / merged['aantal_polissen']
        merged = merged.sort_values(['relatie_nummer', 'subbranche', 'jaar'])
        #shift and min_per_polis
        merged['ewa_min_per_polis'] = (
            merged
            .groupby(['relatie_nummer', 'subbranche'])['minuten_per_polis']
            .transform(lambda x: x.shift(1, fill_value=0)
                                    .ewm(alpha=alpha, adjust=False)
                                    .mean())
        )
        ewm = merged.pivot_table(
            index=['relatie_nummer', 'jaar'],
            columns='subbranche',
            values='ewa_min_per_polis',
            fill_value=0
        ).reset_index()
        ewm.columns.name = None
        ewm = ewm.rename(
            columns=lambda x: f"ewa_min_per_polis_subbranche_{x}" if x not in ['relatie_nummer', 'jaar'] else x
        )
        return ewm

    ewm_df = maak_ewm_minuten_per_polis_subbranche(df)
    result = result.merge(ewm_df, on=['relatie_nummer', 'jaar'], how='left')

    #missings
    result['relatie_nummer'] = result['relatie_nummer'].astype(int)
    result.fillna(0, inplace=True)

    return result


Dekking polis nodig voor subbranche

In [5]:
#DEKKING #basis cleaning in progress
eda_basis(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Polissen_actief_coass_dekking.txt", "df_dekking",[1])
df_dekking.columns = df_dekking.columns.str.replace('\t', '')
df_dekking = df_dekking.drop(columns=["product_", "relatie_naam", "relatie_extern_relatienummer", "relatie_kvk_nummer", "dekking_desc", "hoofdrelatie_naam", "producent", "kantoormedewerker"])
df_dekking['polis_jaarpremie'] = df_dekking['polis_jaarpremie'].str.replace('€', '').str.replace('.', '').str.replace(',', '.').astype(float)


In [6]:
#royement #basis cleaning in progress
eda_basis(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Polissen_royement_2016-2025.txt", "df_roy",[1])
df_roy.columns = df_roy.columns.str.replace('\t', '')  

df_roy = df_roy[['polis', 'subbranche', 'coassurantiemaatschappij_soort']]
print(df_roy.columns )


Index(['polis', 'subbranche', 'coassurantiemaatschappij_soort'], dtype='object')


In [7]:
x = df_dekking
x = x[~x["relatie_nummer"].isin([-3])]
df_dekking = x
df_maatschappij_subbranche = df_dekking[['polis', 'subbranche', 'coassurantiemaatschappij_soort' ]].drop_duplicates(subset='polis')
df_maatschappij_subbranche = pd.concat(
    [df_maatschappij_subbranche, df_roy],
    axis=0,
    ignore_index=True)


tijdregistratie: df_tijdregistratie

In [8]:
paden = [
    r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Uren_2023-2025.txt",
    r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Uren_2020-2022.txt",
    r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Uren_2018-2019.txt",
    r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Uren_2016-2017.txt"
]


alle_results = [verwerk_urenbestand(pad) for pad in paden]
df_tijdregistratie = pd.concat(alle_results, ignore_index=True)
df_tijdregistratie['relatie_nummer'] = df_tijdregistratie['relatie_nummer'].astype(int)

Index(['hoofdrelatie_nummer', 'relatie_nummer', 'polis_x', 'gekoppeld_dossier',
       'gekoppelde_product_\t', 'gekoppelde_product_code', 'gekoppelde_polis',
       'schadegeval', 'soort_uren', 'dienst', 'werksoort', 'afdeling_dvo',
       'soort_dvo', 'datum', 'minuten', 'polis_y', 'subbranche',
       'coassurantiemaatschappij_soort', 'jaar'],
      dtype='object')
Index(['hoofdrelatie_nummer', 'relatie_nummer', 'polis_x', 'gekoppeld_dossier',
       'gekoppelde_product_\t', 'gekoppelde_product_code', 'gekoppelde_polis',
       'schadegeval', 'soort_uren', 'dienst', 'werksoort', 'afdeling_dvo',
       'soort_dvo', 'datum', 'minuten', 'polis_y', 'subbranche',
       'coassurantiemaatschappij_soort', 'jaar'],
      dtype='object')
Index(['hoofdrelatie_nummer', 'relatie_nummer', 'polis_x', 'gekoppeld_dossier',
       'gekoppelde_product_\t', 'gekoppelde_product_code', 'gekoppelde_polis',
       'schadegeval', 'soort_uren', 'dienst', 'werksoort', 'afdeling_dvo',
       'soort_dvo', 'dat

In [9]:
display(df_tijdregistratie.isna().sum())

relatie_nummer                                                       0
jaar                                                                 0
totaal_minuten                                                       0
frequentie_taakregistratie                                           0
schade                                                               0
unieke_werksoort                                                     0
unieke_dienst                                                        0
unieke_gekoppeld_product                                             0
unieke_soort_dvo                                                     0
unieke_afdeling_dvo                                                  0
unieke_soort_uren                                                    0
minuten_soort_dvo_ADVIES                                             0
minuten_soort_dvo_BEHEER                                             0
minuten_soort_dvo_CLAIM                                              0
minute

#verwerking relatie info: df_klantinfo

In [10]:
df = pd.read_csv(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Relatie_info.txt", sep=',')
df = df.drop(columns=['Relatie Naam', 'Hoofdrelatie Naam', 'Relatie Extern Relatienummer', 'Relatie KvK Nummer', 'Idc Relatie Entity', 'SBI DESC', 'Correspondentie Land', "Aantal Relaties", "Status Relatie", "Afdeling"])
df.columns = [c.replace(" ", "_").lower() for c in df.columns]
df = df.drop_duplicates(subset='relatie_nummer')
#df_klantinfo = df.rename(columns={ "correspondentie_woonplaats" : "plaats", "Aantal Werknemers" : "FTE"})
df['sbi_code'] = df['sbi_code'].replace(['_^_', '_~_'], np.nan)
#df['sbi_code'] = df['sbi_code'].str.zfill(5)
df_klantinfo = df
dochter_counts = df_klantinfo.groupby('hoofdrelatie_nummer')['relatie_nummer'].nunique()
df_klantinfo['aantal_dochters'] = df_klantinfo.apply(
    lambda row: dochter_counts[row['hoofdrelatie_nummer']]-1
    if row['hoofdrelatie_nummer'] == row['relatie_nummer']
    #else pd.NA,
    else 0,
    axis=1
)

#display(df_klantinfo.head(5))
#display(df_klantinfo.dtypes)


C:\Users\UNMAWI\AppData\Local\Temp\ipykernel_3568\379548748.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Relatie_info.txt", sep=',')


In [11]:

df_klant_en_tijd = df_tijdregistratie.merge(df_klantinfo, on='relatie_nummer', how='left')
# Zet relatie_sinds om naar datetime
df_klant_en_tijd['relatie_sinds'] = pd.to_datetime(df_klant_en_tijd['relatie_sinds'], format='%d-%m-%Y', errors='coerce')

# Haal het jaartal eruit
df_klant_en_tijd['relatiejaar'] = df_klant_en_tijd['relatie_sinds'].dt.year
# Bereken klantduur
df_klant_en_tijd['klantduur'] = df_klant_en_tijd['jaar'] - df_klant_en_tijd['relatiejaar']



In [12]:
df_klant_en_tijd = df_klant_en_tijd[df_klant_en_tijd['soort_relatie'] == 'Z']
unieke_waarden = df_klant_en_tijd.nunique()
aantal_nan = df_klant_en_tijd.isna().sum()
percent_nan = (aantal_nan / len(df_klant_en_tijd) * 100).round(2)
kolomtypes = df_klant_en_tijd.dtypes

overzicht = pd.DataFrame({
    'type': kolomtypes,
    'unieke_waarden': unieke_waarden,
    'aantal_NaN': aantal_nan,
    'percentage_NaN': percent_nan.astype(str) + '%'
})

display(overzicht)

,type,unieke_waarden,aantal_NaN,percentage_NaN
relatie_nummer,int32,28009,0,0.0%
jaar,int32,10,0,0.0%
totaal_minuten,float64,6268,0,0.0%
frequentie_taakregistratie,int64,911,0,0.0%
schade,int64,150,0,0.0%
unieke_werksoort,int64,55,0,0.0%
unieke_dienst,int64,27,0,0.0%
unieke_gekoppeld_product,int64,24,0,0.0%
unieke_soort_dvo,int64,4,0,0.0%
unieke_afdeling_dvo,int64,7,0,0.0%


Polis 

In [13]:
#premiestroom #basis cleaning in progress
eda_basis(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Polissen_actief_coass_premiestroom.txt", "df_premiestroom",[1])
df_premiestroom.columns = df_premiestroom.columns.str.replace('\t', '')
df_premiestroom = df_premiestroom.drop(columns=["product_", "relatie_naam", "relatie_extern_relatienummer", "relatie_kvk_nummer", "hoofdrelatie_naam", "producent", "kantoormedewerker"])

In [14]:
#x = df_premiestroom
#x = x[~x["relatie_nummer"].isin([-3])]
#df_premiestroom = x
df_premiestroom= df_premiestroom.rename(columns={"polis_jaarpremie": "assurantiebelasting", 
                                          "polis_jaarpremie.1": "afsluit", 
                                          "polis_jaarpremie.2": "poliskosten", 
                                          "polis_jaarpremie.3": "doorloop", 
                                          "polis_jaarpremie.4": "volmachtbeloning", 
                                          "polis_jaarpremie.5": "premie", 
                                          "polis_jaarpremie.6": "do", 
                                          "polis_jaarpremie.7": "totaal"})



In [15]:
#royement #basis cleaning in progress
eda_basis(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Polissen_royement_2016-2025.txt", "df_royement",[1,2])
df_royement.columns = df_royement.columns.str.replace('\t', '')
df_royement = df_royement.drop(columns=["product_", "relatie_naam", "relatie_kvk_nummer", "hoofdrelatie_naam", "producent", "kantoormedewerker"])


In [16]:
#x = df_royement
#x = x[~x["relatie_nummer"].isin([-3])]
#df_royement = x
df_royement = df_royement.rename(columns={"totale_polis_jaarpremie": "kortingen", 
                                          "totale_polis_jaarpremie.1": "assurantiebelasting", 
                                          "totale_polis_jaarpremie.2": "afsluit", 
                                          "totale_polis_jaarpremie.3": "poliskosten", 
                                          "totale_polis_jaarpremie.4": "doorloop", 
                                          "totale_polis_jaarpremie.5": "volmachtbeloning", 
                                          "totale_polis_jaarpremie.6": "premie", 
                                          "totale_polis_jaarpremie.7": "totaal"})


Docu's en processen

In [17]:
#processen #basis cleaning in progress
eda_basis(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Processen_activiteiten_per_relatie_per_jaar.txt", "df_processen",[1])
df_processen = df_processen.drop(columns=["relatie", "unnamed:_2"]).rename(columns={"unnamed:_1" : "relatie_nummer"})

In [18]:
#documenten ge aggreerd op jaar basis
eda_basis(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Aantal_documenten_per_relatie_per_maand.txt", "df_documenten")
df_documenten = df_documenten.drop(columns=["relatie_naam", "relatie_extern_relatienummer"])
#display(df_documenten.head())
df_documenten[['jaar', 'maand']] = df_documenten['maand_telling'].astype(str).str.split('-', expand=True)
df_documenten['jaar'] = df_documenten['jaar'].astype(int)
df_documenten['maand'] = df_documenten['maand'].astype(int)
df_documenten = df_documenten.drop(columns=["maand_telling"])
#display(df_documenten.dtypes)

In [19]:

df_documenten_jaar = df_documenten.groupby(['relatie_nummer', 'jaar']).agg({'aantal_documenten_slimdossier': 'sum'}).reset_index()


In [20]:
df_pros_doc = pd.merge(df_processen, df_documenten_jaar, on=['relatie_nummer', 'jaar'], how='left')
df_pros_doc['relatie_nummer'] = df_pros_doc['relatie_nummer'].astype(int)

Objecten per rela

In [21]:
df = pd.read_excel(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Zakelijke Relaties Objecten v20250425.xlsx")
df.columns = [c.replace(" ", "_").lower() for c in df.columns]
df = df.rename(columns={'relatienummer': 'relatie_nummer'})
df = df.drop(columns=["bedrijfnummer", "vrije_relatie_rubriek_01"])
df = df.drop(index=0).reset_index(drop=True)



In [22]:

df_objecten = df.groupby(['relatie_nummer']).agg({'aantal_objecten':'sum'}).reset_index()


premie en verzekerdbedrag

In [23]:
df_raw = pd.read_csv(
    r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Relatie Netto Premie en Verzekerd Bedrag 2022-2025 per maand.txt",
    header=None
)

#headerfix
mask = df_raw.iloc[0:5].astype(str).apply(
    lambda col: col.str.contains('Polis Jaarpremie', case=False, na=False)
).any()

#V
identificatie_kolommen = [0, 1, 2]
premie_colnames = df_raw.columns[mask].tolist()
houden_kolommen = identificatie_kolommen + premie_colnames

df_premie = df_raw[houden_kolommen].copy()
df_premie.columns = df_premie.iloc[0]  # rij 0 = header
df_premie = df_premie.drop(index=[0, 1, 2])  # verwijder rij 0 (nu header), en rij 1–2


kolomnamen = df_premie.columns.tolist()
kolomnamen[0:3] = ['relatie', 'relatie_nummer', 'polisnummer']
df_premie.columns = kolomnamen


df_premie = df_premie.drop(columns=['relatie', 'polisnummer'])


for col in df_premie.columns[1:]:
    df_premie[col] = (
        df_premie[col]
        .astype(str)
        .str.replace('€', '', regex=False)
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
    )
    df_premie[col] = pd.to_numeric(df_premie[col], errors='coerce')


#Wide to long
df_premie_long = df_premie.melt(id_vars=['relatie_nummer'], var_name='periode', value_name='waarde')

#Split
df_premie_long['jaar'] = df_premie_long['periode'].str[:4].astype(int)
df_premie_long['maand'] = df_premie_long['periode'].str[5:].astype(int)

#Sort
df_premie_long = df_premie_long[['relatie_nummer', 'jaar', 'maand', 'waarde']].sort_values(by=['relatie_nummer', 'jaar', 'maand'])
df_premie_long = df_premie_long.rename(columns={'waarde': 'netto_premie'})
df_premie = df_premie_long
#Reset
df_premie = df_premie.reset_index(drop=True)

C:\Users\UNMAWI\AppData\Local\Temp\ipykernel_3568\545948195.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,78,80,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(


In [25]:
#Lees het bestand volledig in
df_raw = pd.read_csv(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Relatie Netto Premie en Verzekerd Bedrag 2022-2025 per maand.txt",
    header=None
)


mask = df_raw.iloc[0:5].astype(str).apply(
    lambda col: col.str.contains('Verzekerd Bedrag Maatschappij', case=False, na=False)
).any()

#Headerfix
identificatie_kolommen = [0, 1, 2]
verzk_colnames = df_raw.columns[mask].tolist()
alle_te_houden_kolommen = identificatie_kolommen + verzk_colnames

df_verzekerd = df_raw[alle_te_houden_kolommen].copy()

df_verzekerd.columns = df_verzekerd.iloc[0]  # rij 0 = header
df_verzekerd = df_verzekerd.drop(index=[0, 1, 2])  # verwijder rij 0 (nu header), rij 1 en 2

kolomnamen = df_verzekerd.columns.tolist()
kolomnamen[0:3] = ['relatie', 'relatie_nummer', 'polisnummer']
df_verzekerd.columns = kolomnamen

#Alwhat to keep
df_verzekerd = df_verzekerd.drop(columns=['relatie', 'polisnummer'])

#clean numericals
for col in df_verzekerd.columns[1:]:  # vanaf kolom 1, want 0 = relatie_nummer
    df_verzekerd[col] = (
        df_verzekerd[col]
        .astype(str)
        .str.replace('.', '', regex=False)
    )
    df_verzekerd[col] = pd.to_numeric(df_verzekerd[col], errors='coerce')
# wide to long
df_long = df_verzekerd.melt(id_vars=['relatie_nummer'], var_name='periode', value_name='waarde')


df_long['jaar'] = df_long['periode'].str[:4].astype(int)
df_long['maand'] = df_long['periode'].str[5:].astype(int)
#sort
df_long = df_long[['relatie_nummer', 'jaar', 'maand', 'waarde']].sort_values(by=['relatie_nummer', 'jaar', 'maand'])
df_long = df_long.rename(columns={'waarde': 'verzekerdbedrag'})
df_verzekerd = df_long
#Reset index
df_verzekerd = df_verzekerd.reset_index(drop=True)



C:\Users\UNMAWI\AppData\Local\Temp\ipykernel_3568\1493464289.py:2: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,78,80,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(r"C:\Users\UNMAWI\OneDrive - VLC & Partners\Documenten\AAAAAAAAAAAAAScriptie\Scritpie+data\B\Relatie Netto Premie en Verzekerd Bedrag 2022-2025 per maand.txt",


In [26]:
df_premie_verzekerd = df_verzekerd.merge(df_premie, on=['relatie_nummer', 'jaar', 'maand'], how='left')
df_premie_verzekerd['relatie_nummer'] = df_premie_verzekerd['relatie_nummer'].astype(int)

In [27]:
def ewa_stand_per_jaar(df, kolom, span=3):
    df = df.copy()
    
    if kolom == 'netto_premie':
        df[kolom] = df[kolom].clip(lower=0)  # negatieve premies op 0 zetten
    
    df = df.sort_values(['relatie_nummer', 'jaar', 'maand'])
    ewa_result = (
    df.groupby(['relatie_nummer', 'jaar'], group_keys=False)
      .apply(lambda g: g[kolom].ewm(span=span, adjust=False).mean().iloc[-1])
      .reset_index(name=f'ewa_{kolom}')
    )

    return ewa_result
    

In [28]:
ewa_verzekerd = ewa_stand_per_jaar(df_premie_verzekerd, 'verzekerdbedrag', span=3)
ewa_premie = ewa_stand_per_jaar(df_premie_verzekerd, 'netto_premie', span=3)

ewa_combined = ewa_verzekerd.merge(ewa_premie, on=['relatie_nummer', 'jaar'])


C:\Users\UNMAWI\AppData\Local\Temp\ipykernel_3568\3843622977.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g[kolom].ewm(span=span, adjust=False).mean().iloc[-1])
C:\Users\UNMAWI\AppData\Local\Temp\ipykernel_3568\3843622977.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g[kolom].ewm(span=span, adjust=False).mean().iloc[-1])


Data sets'

datasetcombi's

In [29]:

#klant_entijd met _processen, en documenten
df_combi = pd.merge(df_klant_en_tijd, df_pros_doc, on=['relatie_nummer', 'jaar'], how='left')
df_combi = df_combi.drop(columns=['soort_relatie', 'hoofdrelatie_nummer'])
#combi met objecten
df_combi = df_combi.merge(df_objecten, on='relatie_nummer', how='left')
#combi met dekking en premie
df_combi = df_combi.merge(ewa_combined, on=['relatie_nummer', 'jaar'], how='left')

In [30]:
display(df_combi.dtypes)

relatie_nummer                                                           int32
jaar                                                                     int32
totaal_minuten                                                         float64
frequentie_taakregistratie                                               int64
schade                                                                   int64
unieke_werksoort                                                         int64
unieke_dienst                                                            int64
unieke_gekoppeld_product                                                 int64
unieke_soort_dvo                                                         int64
unieke_afdeling_dvo                                                      int64
unieke_soort_uren                                                        int64
minuten_soort_dvo_ADVIES                                               float64
minuten_soort_dvo_BEHEER                            

encode relatie_nummer voor privacy

In [31]:
import uuid

# Functie om een UUID te genereren voor relatie_nummer
def generate_uuid(value):
    # Zet de integer om naar een string voordat de UUID wordt gegenereerd
    value_str = str(value)
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, value_str))

# Maak een kopie van het dataframe om te werken
df_final = df_combi.copy()

# Relatie nummer coderen met een UUID (de UUID blijft consistent per relatie_nummer)
df_final['relatie_nummer_uuid'] = df_final['relatie_nummer'].apply(generate_uuid)

# Verwijder de originele relatie_nummer kolom om privacy te waarborgen
df_final = df_final.drop(columns=['relatie_nummer'])

# Bekijk de eerste paar regels om te verifiëren
print(df_final[['relatie_nummer_uuid']].head())

# Sla het resultaat op in een CSV-bestand
df_final.to_csv('df_final.csv', index=False)

# Optioneel: Bekijk de shape van het nieuwe dataframe
print(f"✅ df_final is succesvol aangemaakt: {df_final.shape[0]} regels en {df_final.shape[1]} kolommen.")


                    relatie_nummer_uuid
0  6ed955c6-506a-5343-9be4-2c0afae02eef
1  a6c4fc8f-6950-51de-a9ae-2c519c465071
2  a6c4fc8f-6950-51de-a9ae-2c519c465071
3  a6c4fc8f-6950-51de-a9ae-2c519c465071
4  e99caacd-6c45-5906-bd9f-b79e62f25963
✅ df_final is succesvol aangemaakt: 96166 regels en 111 kolommen.


In [32]:
df_final.shape

(96166, 111)